In [1]:
import cv2 as cv

In [12]:
video = cv.VideoCapture('my_video.mp4')

In [14]:
# Display relative video propreties
print("Video frame width: \t  ", video.get(cv.CAP_PROP_FRAME_WIDTH))
print("Video frame height: \t  ", video.get(cv.CAP_PROP_FRAME_HEIGHT))
print("Video frame rate property:", video.get(cv.CAP_PROP_FPS))

Video frame width: 	   854.0
Video frame height: 	   480.0
Video frame rate property: 23.976023976023978
